### `Трансформер`

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm import tqdm # отображать загрузку
import torch

In [2]:
df = pd.read_csv('authors.csv')

In [3]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

Из моделей трансформеров с hugging face, я выбирала среди Bert, Roberta и DistilBert. <br>
У всех примерно схожий функционал, однако DistilBert занимает меньше времени для обучения, <br>
и требует меньше места в памяти, что подходит для меня в условиях ограниченной компьютерной мощности...

In [4]:
# загрузка модели
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# предобработка
def preprocess(data, max_length=128):
    tokenized = tokenizer(list(data["text"]), max_length=max_length, padding="max_length", truncation=True, return_tensors="pt") # макс --> если короче --> если длиннее
    labels = torch.tensor(list(data["index"]))
    return tokenized, labels

In [6]:
train_data, train_labels = preprocess(train_df)
test_data, test_labels = preprocess(test_df)

In [7]:
train_dataset = TensorDataset(train_data["input_ids"], train_data["attention_mask"], train_labels)
test_dataset = TensorDataset(test_data["input_ids"], test_data["attention_mask"], test_labels)

In [8]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # где обрабатывается моделька, GPU считается более эффективной, поэтому её первой чекаем
model.to(device)
optimizer = AdamW(model.parameters(), lr=0.00005, no_deprecation_warning=True) # Adam with Weight decay, от переобучения
loss_count = torch.nn.CrossEntropyLoss()

In [10]:
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"): #визуализация загрузки
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_count(outputs.logits, labels)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Average Loss: {average_loss:.4f}")

Epoch 1/3: 100%|██████████████████████████████████████████████████████████████████████| 478/478 [32:43<00:00,  4.11s/it]


Epoch 1/3, Average Loss: 0.3026


Epoch 2/3: 100%|██████████████████████████████████████████████████████████████████████| 478/478 [31:37<00:00,  3.97s/it]


Epoch 2/3, Average Loss: 0.2238


Epoch 3/3: 100%|██████████████████████████████████████████████████████████████████████| 478/478 [31:06<00:00,  3.90s/it]

Epoch 3/3, Average Loss: 0.2042


In [11]:
model.eval()
predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1) # max возвращает values и id, нужны только вторые
        predictions.extend(predicted.cpu().numpy())

Evaluating: 100%|█████████████████████████████████████████████████████████████████████| 120/120 [02:24<00:00,  1.20s/it]


In [25]:
accuracy = accuracy_score(test_labels.numpy(), predictions)
print(f"Accuracy: {accuracy*100:.4f}")

Accuracy: 89.7436


In [19]:
def predict_class(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    
    with torch.no_grad():
        model.eval()
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted_class = torch.max(outputs.logits, 1)

    return predicted_class.item()

In [21]:
remark_text ='Женщина шла наискосок через мост прямо на Равика. Она шла  быстро,  но каким-то  нетвердым шагом. Равик заметил ее лишь  тогда, когда она оказалась почти  рядом.'
prediction = predict_class(remark_text)
print(f"Класс: {prediction}")

Класс: 0


In [22]:
my_text = 'Ввожу текст написанный мной для того чтоб оценить точность модельки'
prediction = predict_class(my_text)
print(f"Класс: {prediction}")

Класс: 0


In [23]:
krapivin_text = 'Револьвер был итальянской системы «Пикколо». В точности как настоящий. Только вместо медных патронов – стеклянные баллончики со сжатым воздухом. Хлопал он оглушительно, бил пластмассовыми пулями крепко, но застрелиться из него было все-таки невозможно. Я и не пробовал. Вместо этого лежал на тахте и стрелял по фужерам.'
prediction = predict_class(krapivin_text)
print(f"Класс: {prediction}")

Класс: 1


### Results and Comparison
1) `Классика`: легка в имплементации, занимает мало времени, высокий результат accuracy. <br>
Однако для больших датасетов и сложных тз может не подойти из-за своей простоты.
2) `Нейронка`: даёт больше контроля над архитектурой модели, чем классика, занимает мало времени, accuracy тоже высокая. <br>
3) `Transformer`: уже pretrained, можно fine-tune для конкретного задания. Accuracy тоже достаточно высокая. <br>
Занимает гораздо больше времени, требует больше computer resources.